In [18]:
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
from mne.io import read_raw_ctf, read_raw_fif
from matplotlib.pyplot import plot as plt
import mne
import numpy as np
from scipy.io import loadmat
import pandas as pd
import os
from IPython.display import clear_output
import time
%matplotlib notebook
%autosave 30

Autosaving every 30 seconds


In [19]:
folderpath = '/home/karim/DATA/DATAmeg_gradCPT/'
savepath = folderpath + 'saflow_preproc/'
subfolder = "20190807/"
filepath = folderpath + subfolder


In [20]:
def find_rawfile(subj, bloc, all_files):
        
    ### Find the right logfile for a specific subject and bloc in a list of log_files
    # (typically the list of files in the log folder, obtained by "os.listdir(LOGS_DIR)")
    for file in all_files:
        if 'SAflow-yharel' in file and '.ds' in file:
            if file[2:4] ==  subj and file[-4] == str(int(bloc)+1):
                break
    return file



def saflow_PP(filename):
    subject_id = filename[:4]
    filepath = folderpath + subfolder + '/' + filename
    bloc_id = filename[-5:-3] ### here change for a loop
    raw_data = read_raw_ctf(filepath, preload=True)
    picks = mne.pick_types(raw_data.info, meg=True, eog=True, exclude='bads')
    raw_data.plot();
    raw_data.plot_psd(average=False, picks=picks);

    ## Filtering
    high_cutoff = 200
    low_cutoff = 0.5
    raw_data.filter(low_cutoff, high_cutoff, fir_design="firwin")
    raw_data.notch_filter(np.arange(60, high_cutoff+1, 60), picks=picks, filter_length='auto',phase='zero', fir_design="firwin")
    raw_data.plot_psd(average=False, picks=picks);

    ## ICA
    ica = ICA(n_components=20, random_state=0).fit(raw_data, decim=3)
    ica.plot_sources(raw_data);
    fmax = 40. ## correlation threshold for ICA components (maybe increase to 40. ?)

    ## FIND ECG COMPONENTS
    ecg_epochs = create_ecg_epochs(raw_data, ch_name='EEG059')
    ecg_inds, ecg_scores = ica.find_bads_ecg(ecg_epochs, ch_name='EEG059')
    ica.plot_scores(ecg_scores, ecg_inds); 
    ica.plot_properties(ecg_epochs, picks=ecg_inds, psd_args={'fmax': fmax}, image_args={'sigma': 1.});

    ## FIND EOG COMPONENTS
    eog_epochs = create_eog_epochs(raw_data, ch_name='EEG057')
    eog_inds, eog_scores = ica.find_bads_eog(eog_epochs, ch_name='EEG057')
    ica.plot_scores(eog_scores, eog_inds);
    ica.plot_properties(eog_epochs, picks=eog_inds, psd_args={'fmax': fmax}, image_args={'sigma': 1.});

    ## EXCLUDE COMPONENTS
    ica.exclude = ecg_inds
    ica.apply(raw_data)
    ica.exclude = eog_inds
    ica.apply(raw_data)
    raw_data.plot(); # Plot the clean signal.

    ## SAVE PREPROCESSED FILE
    time.sleep(60)
    raw_data.save(savepath + subject_id + '_' + bloc_id + '_preprocessed.fif.gz', overwrite=True)
    time.sleep(30)
    #!jupyter nbconvert saflow_preproc.ipynb --output $filename
    %notebook savepath+subject_id+'_'+bloc_id+'_preproclog.ipynb'
    clear_output()

In [21]:
subj_list = ['09']
blocs_list = ['1', '2', '3', '4', '5', '6']
all_files = os.listdir(filepath)

for subj in subj_list:
    for bloc in blocs_list:
        file = find_rawfile(subj, bloc, all_files)
        saflow_PP(file)

In [15]:
#### CREATE A SCRIPT THAT RUNS THE PREPROC AND SAVES A REPORT FILE